In [11]:
import json
import requests
from typing import List, Dict
from decimal import Decimal

def check_wallet_balance(address: str, token_id: str) -> bool:
    """
    Check if an address holds any amount of a specific token.
    
    Args:
        address (str): Ergo wallet address
        token_id (str): Token ID to check for
    
    Returns:
        bool: True if wallet holds any amount of the token
    """
    try:
        response = requests.get(f"https://api.ergoplatform.com/api/v1/addresses/{address}/balance/confirmed")
        response.raise_for_status()
        
        balance_data = response.json()
        tokens = balance_data.get('tokens', [])
        
        return any(token['tokenId'] == token_id for token in tokens)
    except Exception as e:
        print(f"Error checking balance for {address}: {str(e)}")
        return False

def fetch_miners_data(block_heights):
    """
    Fetch miners data from the API.
    
    Args:
        block_heights (list): List of block heights to query
    
    Returns:
        dict: JSON response from the API
    """
    base_url = "http://5.78.102.130:8000/sigscore/miners/average-participation"
    
    # Convert block heights list to comma-separated string
    blocks_param = ",".join(map(str, block_heights))
    
    # Make the API request
    response = requests.get(f"{base_url}?blocks={blocks_param}")
    response.raise_for_status()  # Raise exception for bad status codes
    
    return response.json()

def generate_rights_distribution(miners_data: Dict, 
                               total_amount: float, 
                               emission_token_name: str,
                               rights_token_id: str) -> Dict:
    """
    Generate distribution only for miners holding the rights token.
    
    Args:
        miners_data (dict): Miner participation data
        total_amount (float): Total emission tokens to distribute
        emission_token_name (str): Name of token being distributed
        rights_token_id (str): Token ID that miners must hold
    """
    eligible_miners = []
    total_eligible_percentage = Decimal('0')
    
    # First pass: identify eligible miners
    for miner in miners_data['miners']:
        if check_wallet_balance(miner['miner_address'], rights_token_id):
            eligible_miners.append(miner)
            total_eligible_percentage += Decimal(str(miner['avg_participation_percentage']))
    
    # Recalculate percentages based on eligible miners only
    recipients = []
    if total_eligible_percentage > 0:
        for miner in eligible_miners:
            # Adjust percentage relative to eligible total
            adjusted_percentage = (Decimal(str(miner['avg_participation_percentage'])) / 
                                 total_eligible_percentage * 100)
            
            # Calculate amount based on adjusted percentage
            amount = round(float(Decimal(str(total_amount)) * (adjusted_percentage / Decimal('100'))), 8)
            
            if amount > 0:
                recipients.append({
                    "address": miner['miner_address'],
                    "amount": amount
                })
    
    return {
        "distributions": [
            {
                "token_name": emission_token_name,
                "recipients": recipients
            }
        ]
    }

def save_distribution_json(distribution: Dict, output_file: str) -> None:
    with open(output_file, 'w') as f:
        json.dump(distribution, f, indent=2)

# Example usage:
if __name__ == "__main__":
    # Configuration
    BLOCK_HEIGHTS = [1417531]  # Replace with actual block heights
    TOTAL_EMISSION = 10000.0  # Total emission tokens to distribute
    EMISSION_TOKEN_NAME = "FlameShards"
    RIGHTS_TOKEN_ID = "c31e95f0a332288a81839b6e7a227d4fa0d956b36be5f77a1998116352b24947"
    OUTPUT_FILE = "MRP.json"
    
    # Execute distribution flow
    miners_data = fetch_miners_data(BLOCK_HEIGHTS)
    distribution = generate_rights_distribution(
        miners_data,
        TOTAL_EMISSION,
        EMISSION_TOKEN_NAME,
        RIGHTS_TOKEN_ID
    )
    save_distribution_json(distribution, OUTPUT_FILE)

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from typing import List, Dict, Tuple
import seaborn as sns

def check_wallet_balance(address: str, token_id: str) -> bool:
    try:
        response = requests.get(f"https://api.ergoplatform.com/api/v1/addresses/{address}/balance/confirmed")
        response.raise_for_status()
        return any(token['tokenId'] == token_id for token in response.json().get('tokens', []))
    except Exception as e:
        print(f"Error checking balance for {address}: {str(e)}")
        return False

def analyze_distribution(miners_data: Dict, rights_token_id: str) -> Tuple[List, List, List, List]:
    eligible_addresses = []
    eligible_participations = []
    ineligible_addresses = []
    ineligible_participations = []
    
    for miner in miners_data['miners']:
        has_token = check_wallet_balance(miner['miner_address'], rights_token_id)
        if has_token:
            eligible_addresses.append(miner['miner_address'][:8] + '...')
            eligible_participations.append(miner['avg_participation_percentage'])
        else:
            ineligible_addresses.append(miner['miner_address'][:8] + '...')
            ineligible_participations.append(miner['avg_participation_percentage'])
            
    return (eligible_addresses, eligible_participations, 
            ineligible_addresses, ineligible_participations)

def plot_distribution_analysis(miners_data: Dict, rights_token_id: str):
    # Set style
    # plt.style.use('seaborn')
    fig = plt.figure(figsize=(15, 10))
    
    # Get data
    (eligible_addrs, eligible_parts, 
     ineligible_addrs, ineligible_parts) = analyze_distribution(miners_data, rights_token_id)
    
    # Plot 1: Pie chart of eligible vs ineligible miners
    ax1 = plt.subplot(221)
    total = len(eligible_addrs) + len(ineligible_addrs)
    labels = ['With Rights Token', 'Without Rights Token']
    sizes = [len(eligible_addrs), len(ineligible_addrs)]
    percentages = [s/total*100 for s in sizes]
    colors = ['#2ecc71', '#e74c3c']
    
    ax1.pie(sizes, labels=[f'{l}\n({p:.1f}%)' for l, p in zip(labels, percentages)],
            colors=colors, autopct='%1.1f%%', startangle=90)
    ax1.set_title('Distribution of Miners by Token Status')
    
    # Plot 2: Participation distribution
    ax2 = plt.subplot(222)
    sns.boxplot(data=[eligible_parts, ineligible_parts], 
                ax=ax2, palette=['#2ecc71', '#e74c3c'])
    ax2.set_xticklabels(['Eligible', 'Ineligible'])
    ax2.set_title('Participation Distribution')
    ax2.set_ylabel('Participation Percentage')
    
    # Plot 3: Bar chart of participation by status
    ax3 = plt.subplot(212)
    x = np.arange(max(len(eligible_addrs), len(ineligible_addrs)))
    width = 0.35
    
    if eligible_parts:
        ax3.bar(x[:len(eligible_parts)], eligible_parts, width, 
                label='Eligible', color='#2ecc71', alpha=0.7)
    if ineligible_parts:
        ax3.bar(x[:len(ineligible_parts)] + width, ineligible_parts, width,
                label='Ineligible', color='#e74c3c', alpha=0.7)
    
    ax3.set_ylabel('Participation Percentage')
    ax3.set_title('Individual Miner Participation')
    ax3.set_xlabel('Miner Address')
    ax3.legend()
    
    # Add summary stats
    summary_text = f"""
    Summary Statistics:
    Total Miners: {total}
    Eligible Miners: {len(eligible_addrs)} ({len(eligible_addrs)/total*100:.1f}%)
    Ineligible Miners: {len(ineligible_addrs)} ({len(ineligible_addrs)/total*100:.1f}%)
    
    Eligible Avg Participation: {np.mean(eligible_parts):.2f}%
    Ineligible Avg Participation: {np.mean(ineligible_parts):.2f}%
    """
    plt.figtext(0.02, 0.02, summary_text, fontsize=8, 
                bbox=dict(facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig('distribution_analysis.png', dpi=300, bbox_inches='tight')
    plt.close()

if __name__ == "__main__":
    # Example usage
    BLOCK_HEIGHTS = [1417531]
    RIGHTS_TOKEN_ID = "c31e95f0a332288a81839b6e7a227d4fa0d956b36be5f77a1998116352b24947"
    
    # Fetch data
    response = requests.get(f"http://5.78.102.130:8000/sigscore/miners/average-participation?blocks={','.join(map(str, BLOCK_HEIGHTS))}")
    miners_data = response.json()
    
    # Generate visualization
    plot_distribution_analysis(miners_data, RIGHTS_TOKEN_ID)

/home/whaleshark/.local/lib/python3.10/site-packages/seaborn/_base.py:948: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
/home/whaleshark/.local/lib/python3.10/site-packages/seaborn/categorical.py:632: FutureWarning: SeriesGroupBy.grouper is deprecated and will be removed in a future version of pandas.
  positions = grouped.grouper.result_index.to_numpy(dtype=float)
/home/whaleshark/.local/lib/python3.10/site-packages/seaborn/_base.py:948: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
/home/whaleshark/.local/lib/python3.10/site-packages/seaborn/categorical.py:632: FutureWarning: SeriesGroupBy.grou